## Imports and Config

In [46]:
import os
import time
from dotenv import load_dotenv

import json
import random
from uuid import uuid4

import requests
from ast import literal_eval

import google.generativeai as genai

In [47]:
load_dotenv()

GEMINI_API_KEY = os.environ['GEMINI_API_KEY']

## Trending Topics

In [49]:
url = 'https://trends.google.com/trends/api/dailytrends'
params = {'hl': 'en-US', 'tz': 0, 'geo': 'US'}

response = requests.get(url, params=params)
json_data = literal_eval(response.text[5:])
searches = json_data['default']['trendingSearchesDays'][0]['trendingSearches']

out = {}
for s in searches:
    key = s['title']['query']
    out[key] = {'traffic': s['formattedTraffic']}
    out[key]['articles'] = [{'title': x['title'], 'time_ago': x['timeAgo'], 'snippet': x['snippet']} for x in s['articles']]
    out[key]['related_queries'] = [x['query'] for x in s['relatedQueries']]
out

{'Alabama football': {'traffic': '1M+',
  'articles': [{'title': 'After tough South Carolina win, what&#39;s next for Alabama football in ...',
    'time_ago': '3h ago',
    'snippet': 'When an SEC team comes to Bryant-Denny Stadium to play Alabama, Tim Smith knows exactly what the Crimson Tide will get.'},
   {'title': 'Alabama 27-25 South Carolina (Oct 12, 2024) Game Recap - ESPN',
    'time_ago': '5h ago',
    'snippet': 'Expert recap and game analysis of the Alabama Crimson Tide vs. South Carolina Gamecocks NCAAF game from October 12, 2024 on ESPN.'},
   {'title': 'Grading Alabama football&#39;s ugly South Carolina win',
    'time_ago': '5h ago',
    'snippet': 'UA sealed the game on the final play, when Domani Jackson intercepted a heave from USC&#39;s LaNorris Sellers to preserve the 27-25 victory. A win is a win in the&nbsp;...'},
   {'title': 'No. 7 Alabama Football Survives Close Battle With South Carolina ...',
    'time_ago': '7h ago',
    'snippet': 'TUSCALOOSA, Ala.— Follo

In [50]:
ix = random.randint(0, len(out))
topic = list(out.keys())[ix]
topic

'Poland vs Portugal'

## Text Prompt

In [51]:
prompt = """
You are a social media marketing expert working for a global journalistic firm, tasked with increasing engagement and viewership on Instagram, YouTube, and TikTok. Your goal is to create visually compelling, shareable content, including images, memes, and short videos.

Based on the provided trending topic "{topic}" and region "{country}", generate creative content ideas for social media posts that can go viral. Each idea should be relevant to the local culture and trends in the specified region. Follow these guidelines when generating the ideas:

1. **Categories**: The response should include three main categories:
   - **Video**: A list of 5 video clips, each with a detailed description, text overlay, and voiceover.
   - **Image**: A single image with a description, text overlay, and voiceover.
   - **Meme**: A single meme with a description, text overlay, and voiceover.

2. **Video Content Details**:
   - Provide a list of 5 elements for the video category.
   - Each element should include:
     - **Description**: What the video clip should depict, including visual elements, animations, or transitions.
     - **Text Overlay**: Text that appears on the screen to reinforce the message.
     - **Voiceover**: A short voiceover script that aligns with the visuals and conveys the key message.

3. **Image Content Details**:
   - Describe the content of the image, including visual elements and any regional cultural references.
   - Include a **Text Overlay** that adds context or reinforces the message.
   - Provide a **Voiceover** script that complements the image content.

4. **Meme Content Details**:
   - Describe the meme’s visual elements and the humorous or viral aspect.
   - Include a **Text Overlay** that adds to the meme's humor or message.
   - Provide a **Voiceover** script that adds an extra layer of humor or commentary.

5. **Song**: For all three categories (video, image, and meme), provide a **Song** section containing:
   - **Name**: The name of a trending song relevant to the topic and region.
   - **Artist**: The artist's name. Ensure the song is popular and culturally relevant.

6. **Engagement Strategy**: Incorporate features to encourage engagement, such as hashtags, call-to-actions (e.g., 'tag a friend', 'share your opinion'), or challenges (e.g., a trending dance or meme challenge).

7. **Format for Virality**: Ensure the content design maximizes shares and engagement, utilizing popular social media formats like challenges, polls, reactions, or humor. Avoid content that could evoke negative emotions or controversy.

8. **JSON Formatting Instructions**:
   - Ensure the output is formatted as valid JSON.
   - **Escape any quotes within values** using a backslash (e.g., \\"quote\\").
   - Any nested quotes inside strings should be properly escaped to ensure the JSON is valid and parsable.

Return the response as a properly formatted JSON object containing:
- **video**: A list of 5 dictionaries, where each dictionary contains keys:
  - **description**: Description of the video clip.
  - **text_overlay**: Text that appears on the video.
  - **voiceover**: Voiceover script for the clip.
- **image**: A dictionary containing keys:
  - **description**: Description of the image.
  - **text_overlay**: Text that appears on the image.
  - **voiceover**: Voiceover script for the image.
- **meme**: A dictionary containing keys:
  - **description**: Description of the meme.
  - **text_overlay**: Text that appears on the meme.
  - **voiceover**: Voiceover script for the meme.
- **song**: A dictionary containing:
  - **name**: The song name.
  - **artist**: The artist's name.
Do not include any additional text.
"""


In [52]:
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [53]:
llm_prompt = prompt.format(topic=topic, country='worldwide')
json_text = model.generate_content(llm_prompt).text.lstrip('```json').strip('```')
response = literal_eval(json_text)

In [54]:
response

{'video': [{'description': "A split-screen video showing highlights from Poland and Portugal's recent matches, showcasing the best goals, saves, and moments of skill. Use fast-paced editing with transitions and music to build excitement.",
   'text_overlay': "Poland vs Portugal: Who's got the edge?",
   'voiceover': 'The stage is set for a clash of titans! Poland and Portugal, two footballing giants, ready to battle it out. Witness the electrifying skills, the breathtaking goals, and the ultimate showdown.'},
  {'description': 'A video showing a compilation of fans from both countries reacting to the match, with a mix of excitement, anticipation, and maybe some friendly banter. Use a cheerful and upbeat song.',
   'text_overlay': 'Passion from both sides!',
   'voiceover': "The passion is undeniable! Fans from Poland and Portugal are ready to roar their teams to victory. From chants to cheers, the atmosphere is electric. It's a celebration of football, a global language!"},
  {'descrip

In [55]:
video_details = response['video']
video_images = [media['description'] for media in video_details]
video_texts = [media['text_overlay'] for media in video_details]
video_audios = [media['voiceover'] for media in video_details]

## Image Generation

In [56]:
POLLINATION_URI = 'https://pollinations.ai/p/{prompt}?width=1080&height=1920&seed=42&model=flux'

In [57]:
video_images

["A split-screen video showing highlights from Poland and Portugal's recent matches, showcasing the best goals, saves, and moments of skill. Use fast-paced editing with transitions and music to build excitement.",
 'A video showing a compilation of fans from both countries reacting to the match, with a mix of excitement, anticipation, and maybe some friendly banter. Use a cheerful and upbeat song.',
 'A video showing a funny and light-hearted comparison of Polish and Portuguese culture, like food, dance, or customs. Use humor and exaggeration to keep it light.',
 'A video showing the best moments from past Poland vs Portugal matches, including famous goals, iconic players, and historic rivalries.',
 'A video showcasing the best players from both teams, highlighting their skills and achievements, building anticipation for the match.']

In [58]:
img_ids = []
for i, img in enumerate(video_images):
    res_img = requests.get(POLLINATION_URI.format(prompt=img))
    img_name = f'{i}_{uuid4()}'
    img_ids.append(img_name)
    print(img_name)
    with open(f'{img_name}.jpg', 'wb') as file:
        file.write(res_img.content)
    time.sleep(2)

935a65f5-a9f7-4570-ab49-cc461ed19c24
12dc847e-1651-48e1-aa3a-9b700e0510b5
43eeaa47-660a-4014-9119-7670caa33c84
ebda5a59-5994-40b4-a835-94626c091e17
eeac1e1c-969e-4ff3-871c-681d02835ff7


## Audio Generation